# Investigating changes in mixed layer depth at Sørvest-F  due to wind farms

__Kjersti Stangeland - Summer 2025 - The Norwegian Meteorological Insitute__

#### The project - Havvind

A scenario with Norwegian wind farms at sea has been developed in a collaboration with NVE. The period 2/2-2022 til 30/6-2022 is simulated with MEPS, WaveWatch III, og Norkyst v3. There are to versions of the simulations:
* A control run based of the current situation with no Norwegian wind farms at sea
* An experiment with simulated wind farms.

In the experiments, the only direct parametrization of the windmills are in MEPS. Therefore, effects on waves and ocean cirulation is indirectly seen through changed surface fluxes.

__Details on the wind farms:__
* Height of turbines is 165 m
* Radius 143 m
* 2.5 km distance between turbines
* 6 (?) different farms along the Norwegian coast

__Sørvest F__
* A wind farm south west of Norway
* 1x1500MW (430 km²) + 2x2100MW (600km² + 600 km²)
* 258 turbines


### Mixed layer depth

The mixed layer of the upper ocean is the first meters of the water column where the potential density is fairly mixed. Below this layer, the density increases almost exponentially. The depth of the mixed layer depends on available energy for mixing, i.e. turbulence ad convection.

In [28]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cmocean.cm as cmo
import numpy as np
import xarray as xr
import cartopy 
from datetime import datetime, timedelta
import pandas as pd
from Rossby_deformation.get_turbine_coords import get_turbine_coords
from netCDF4 import Dataset
from roppy import SGrid
import xroms
from cartopy import config
from glob import glob
from Rossby_deformation.density import dens
from scipy.interpolate import griddata
from Mixed_layer_depth.mld import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Starting to play around with a test file

In [14]:
def find_indices_of_point(grid, target_lon, target_lat):
    '''
    Finds the [y, x] indices of grid point closest to a given coordinate point (lon, lat).
    The model grid is lon(y, x), lat(y, x).
    '''
    lon = grid.lon_rho
    lat = grid.lat_rho

    y, x = xroms.argsel2d(lon, lat, target_lon, target_lat)

    return y, x

In [4]:
path = '/lustre/storeB/project/nwp/havvind/hav/results/experiment/EXP-03/norkyst_avg_0001.nc'
ds = xroms.open_netcdf(path)
ds, xgrid = xroms.roms_dataset(ds, include_cell_volume=True)
fid = Dataset(path)
grid = SGrid(fid)

In [5]:
zlevs = np.arange(0,51,1)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(52,102,2), axis=0)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(105,305,5), axis=0)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(520,1020,20), axis=0)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(1050,3050,50), axis=0)

zlevs = zlevs[np.where(zlevs<=np.max(grid.h))]
zlevs = np.array(zlevs)*-1.

In [6]:
# Depth of rho-points: z_r
# Adding zero to end of array (surface)
z_r = np.insert(grid.z_r, grid.z_r.shape[0], values = np.zeros_like(grid.h), axis=0)
# Adding local water depth to beginning of array
z_r = np.insert(z_r, 0, values = -1.*grid.h, axis=0)

In [8]:
salt  = ds.variables['salt'][:]
temp = ds.variables['temp'][:]
mask = grid.mask_rho[:,:]

In [ ]:
for y in range(0, 100):
        for x in range(0, 200): 
            if not mask[y, x]:  # skipping land points
               continue

            # Filtering out local water depth on the zlevs
            # Where zlevs is shallower than z_r -> true
            tmpnz = zlevs[np.where(zlevs[:]>z_r[:,y,x].min())].squeeze()  # z-levels

            if 1:
               t = -1
               # Adding first and last value to each end of the array to match length of z-levels (tmpnz)
               tmpnS = salt[t,:,y,x]; tmpnS = np.append(tmpnS,tmpnS[-1]); tmpnS = np.insert(tmpnS,0,values=tmpnS[0],axis=0)
               tmpnT = temp[t,:,y,x]; tmpnT = np.append(tmpnT,tmpnT[-1]); tmpnT = np.insert(tmpnT,0,values=tmpnT[0],axis=0)
               # Interpolating to z-levels
               tempZ = griddata(z_r[:,y,x], tmpnT[:], tmpnz)
               saltZ = griddata(z_r[:,y,x], tmpnS[:], tmpnz)

               potdens = dens(saltZ, tempZ, np.zeros_like(tempZ)).squeeze()

               #mld = xroms.mld(potdens, xgrid, grid.h, mask)


In [ ]:
potdens

array([1023.86865917, 1023.86804648, 1023.86793067, 1023.86794298,
       1023.86802283, 1023.86813967, 1023.86826656, 1023.86838552,
       1023.86848794, 1023.86857315, 1023.86862524])

In [ ]:
def calc_mld(pot_dens):
    '''
    
    '''
    # the mixed layer depth is the depth where the potential density equals the surface - a threshold
    thres = pot_dens[0] - 0.03  # [kgm^⁻3]

    mld = xr.where(pot_dens > thres)

    return mld

In [ ]:
test_mld = calc_mld(potdens)

TypeError: where() missing 2 required positional arguments: 'x' and 'y'

__To-do__:

- find out how to calculate MLD - whats the best definition.
- read: https://gmd.copernicus.org/articles/16/3849/2023/gmd-16-3849-2023.pdf


'The MLD may be
computed using a threshold change in density or temperature
(BM04), a threshold in density gradient (Dong et al., 2008), a
maximum density gradient (Large et al., 1997), a maximum in the curvature of the density profile (Lorbacher et al., 2006),
or a minimum of the relative variance (Huang et al., 2018) or
based on energetic principles (Reichl et al., 2022).'

' ...the potential density threshold method has
been recommended by Griffies et al. (2016) to compute the
MLD in OMIP and CMIP models, with a threshold value
of 0.03 kgm−3
.'

what about: 

- read the paper well and see if their method can be used
- make a function which opens file, interpolates from s to z, calculates density, calculates mixed layer depth, writes pot dens and MLD to a netcdf file. run in a loop for all files both ref and exp and save as you did for rossby. 
- start with testing on daily files, and when method works, do it on all.
- then import those files into this notebook and visualize it. 
- ex monthly means and differences
- then you dont have to bother with the limiting study area yet, because you can just do it on the whole.

oh! xroms has xroms.seawater.mld! try that